In [1]:
import random
import pygame
import sys
import matplotlib.pyplot as plt
from pygame.locals import *
from collections import defaultdict
import numpy as np
from math import sqrt
import time
import dill

pygame 2.6.1 (SDL 2.28.4, Python 3.9.21)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# 初始化pygame
pygame.init()
screen = pygame.display.set_mode((1000, 1000))
pygame.display.set_caption('Q-Learning    Game')

# clock = pygame.time.Clock()
# fps = 100

# 定义颜色
RED = (255, 0, 0)
BLUE = (0, 0, 255)
GREEN = (0, 255, 0)
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)

q_table = 'qtable_1740133310.pickle'

#图片替代颜色
pygame.display.set_caption("Space Flight") 
icon = pygame.image.load("icon.png") 
pygame.display.set_icon(icon) 
bg = pygame.image.load("bg.png") 
player = pygame.image.load("player.png") 
enemy = pygame.image.load("enemy.png") 
bullet = pygame.image.load("bullet.png") 


class Player:
    def __init__(self):
        self.x = 450
        self.y = 900
        # self.color = RED
        self.Img = pygame.image.load("player.png")
        self.width = 100
        self.height = 100
        self.bullet_cooldown = 0

    def action(self, action):
        # 每次动作都递减冷却计数器
        if self.bullet_cooldown > 0:
            self.bullet_cooldown -= 1

        if action == 'left':
            self.x -= 100
            if self.x < 0:
                self.x = 0
        elif action == 'right':
            self.x += 100
            if self.x > 1000 - self.width:
                self.x = 1000 - self.width
        elif action == 'shoot':
            # 只有在冷却结束时才能射击
            if self.bullet_cooldown <= 0:
                if len(bullets) < 20:
                    bullets.append(Bullet(self))
                    self.bullet_cooldown = 1
        elif action == 'wait':
             pass


class Enemy:
    def __init__(self):
        self.x = random.randint(0, 1000 - 100)
        self.y = 0
        # self.color = BLUE
        self.Img = pygame.image.load("enemy.png")
        self.width = 100
        self.height = 100
        self.speed = 100
        self.dx = 1

    def action(self):
        new_x = self.x + self.dx * self.speed
        if new_x + self.width > 1000 or new_x < 0:
            self.dx *= -1
            self.y += self.height
        self.x = new_x


class Bullet:
    def __init__(self, player):
        self.x = player.x + player.width / 2
        self.y = player.y
        # self.color = GREEN
        self.Img = pygame.image.load("bullet.png")
        self.width = 50
        self.height = 50
        self.speed = 400
        self.active = True

    def action(self):
        if self.active:
            self.y -= self.speed
            if self.y < 0:
                self.active = False

    def hit(self, enemy):
        if self.active and (self.x < enemy.x + enemy.width and
                            self.x + self.width > enemy.x and
                            self.y < enemy.y + enemy.height and
                            self.y + self.height > enemy.y):
            enemies.remove(enemy)
            # 当敌人被击中后重新生成一个新的敌人
            enemies.append(Enemy())
            self.active = False
            return True
        return False


def get_state():
    state = [int(player.x / 50)]  # 状态划分
    for enemy in enemies:
        # state.append(int(enemy.x / 50))
        # state.append(int(enemy.y / 50))
        state.append(int(sqrt((player.x/50 - enemy.x/50)**2+(player.y/50 - enemy.y/50)**2)))
    return tuple(state)


def check_game_over():
    for enemy in enemies:
        if enemy.y + enemy.height > 1000:
            return True, -1000

    for enemy in enemies:
        if (player.x < enemy.x + enemy.width and
                player.x + player.width > enemy.x and
                player.y < enemy.y + enemy.height and
                player.y + player.height > enemy.y):
            return True, -1000

    return False, 0


def show_game_state():
    screen.blit(bg, (0, 0))

    # 绘制玩家
    # pygame.draw.rect(screen, player.color,
    #                  (player.x, player.y,  player.height))#纯颜色
    screen.blit(player.Img, (player.x, player.y))

    # 绘制敌人
    for enemy in enemies:
        # pygame.draw.rect(screen, enemy.color,
        #                  (enemy.x, enemy.y, enemy.width, enemy.height))
        screen.blit(enemy.Img, (enemy.x, enemy.y))
        # 绘制子弹
    for bullet in bullets:
        if bullet.active:
            # pygame.draw.rect(screen, bullet.color,
            #                  (bullet.x, bullet.y, bullet.width, bullet.height))
            screen.blit(bullet.Img, (bullet.x, bullet.y))

            # 绘制击中次数
    font = pygame.font.Font(None, 36)
    text = font.render(f'Hits:  {hits}/6', True, WHITE)
    screen.blit(text, (10, 10))

    # 更新显示
    pygame.display.flip()


# Q-Learning参数设置
alpha = 0.2  # 学习率
gamma = 0.995  # 衰减因子
epsilon = 0.01 # 探索率
EPS_DECAY = 0.9998

# 初始化奖励记录列表
episode_rewards = []
# average_rewards = []
update_steps = 2000 # 每300次更新一次平均奖励

# 初始化Q表
if q_table is None:
    q_table = defaultdict(lambda: {'left': 0, 'right': 1, 'shoot': 2,'wait':3})
else:
    with open(q_table,'rb') as f:
        q_table = dill.load(f)


for episode in range(20001):
    player = Player()
    enemies = [Enemy()]
    bullets = []
    hits = 0  # 新增：击中次数计数器
    # total_reward = 0
    done = False

    if episode % update_steps == 0:
        print(f'episode #{episode}, epsilon: {epsilon}')
        print(f'mean reward: {np.mean(episode_rewards[-update_steps:])}')
        show = True
    else:
        show = False

    episode_reward = 0
    reward = 0
    while not done:
        state = get_state()

        # 选择动作
        if random.random() < epsilon:
            action = random.choice(['left', 'right', 'shoot','wait'])
        else:
            action = max(q_table[state], key=lambda k: q_table[state][k])

        # 执行动作
        player.action(action)

        # 更新子弹和敌人状态
        for bullet in bullets.copy():
            bullet.action()
            for enemy in enemies.copy():
                if bullet.hit(enemy):
                    reward += 500
                    hits += 1  

        for enemy in enemies.copy():
            enemy.action()

            # 检查失败条件
        game_over, penalty = check_game_over()
        if game_over:
            reward += penalty
            done = True

            # 检查胜利条件：当击中6次时胜利
        if hits == 6:
            reward += 3000
            done = True

        if action in ['left', 'right']:
            reward -= 50

            # 计算新的状态和奖励
        new_state = get_state()

        # 更新Q表
        best_next_action = max(q_table[new_state], key=lambda k: q_table[new_state][k])
        q_table[state][action] += alpha * (
                reward + gamma * q_table[new_state][best_next_action] - q_table[state][action])

        # 更新游戏状态
        # show_game_state()

        # 处理pygame事件（防止窗口无响应）
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                sys.exit()

                # 记录每回合奖励
    episode_reward += reward

    episode_rewards.append(reward)

    # 减小探索率（epsilon greedy）
    epsilon *= EPS_DECAY

    # clock.tick(fps) 
    # time.sleep(0.5)

pygame.quit()

episode #0, epsilon: 0.01
mean reward: nan


C:\Users\yangc\.conda\envs\deeplearning\lib\site-packages\numpy\_core\fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\yangc\.conda\envs\deeplearning\lib\site-packages\numpy\_core\_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


episode #2000, epsilon: 0.006702932301946425
mean reward: 4536.275


KeyboardInterrupt: 

In [ ]:
moving_avg = np.convolve(episode_rewards,np.ones((update_steps,))/update_steps,mode='valid')
plt.plot([i for i in range(len(moving_avg))],moving_avg)
plt.xlabel('episode ')
plt.ylabel(f'mean{update_steps}reward')
plt.savefig('train_curve.png')
plt.show()

In [ ]:
with open(f'qtable_{int(time.time())}.pickle','wb') as f:
    dill.dump(q_table,f)